## Table of contents

* [Install necessary packages](#installpkgs)
* [Setup DAS Services](#setup)

<a id='installpkgs'></a>
## Install necessary packages

1. (Optional) Create a python virtual environment
2. Install `das-cli` (python) package
3. Install `hyperon` and `jupyterlab` python package
4. Run jupyter lab
5. Access the JupyterLab frontend in your browser

#### 1. (Optional) Create a python virtual environment
```
python3 -m venv .venv
source .venv/bin/activate
```

#### 2. Install `das-cli` (python) package
For Debian based Linux users:
```
sudo apt -y update
sudo apt -y install wget

wget -O - http://45.77.4.33/apt-repo/setup.sh | sudo bash

sudo apt -y install das-toolbox

# >= 0.5.0
das-cli --version
```

Stay tuned to [this channel](https://chat.singularitynet.io/snet/channels/hyperon-das-library-updates) for new versions of das-cli. We recommend always using the latest available.

You can also run `das-cli` from source (other OS):
```
git clone https://github.com/singnet/das-toolbox.git

cd das-toolbox
git checkout tags/0.5.0

pip install -r das-cli/src/requirements.txt
pip install -e das-cli/src/
```

#### 3. Install hyperon and jupyterlab python package
```
pip install hyperon jupyterlab
```

#### 4. Run jupyter lab
```
jupyter lab --ip=0.0.0.0 --port=8888 --no-browser --allow-root --NotebookApp.token='' --NotebookApp.password=''
```

#### 5. Access the JupyterLab frontend in your browser:
You should be able to access the JupyterLab at:
http://127.0.0.1:8888/lab

<a id='setup'></a>
## Setup DAS Services

DAS Services can be set up using the [DAS Toolbox](https://github.com/singnet/das-toolbox) following these steps:

1. Setup `das-cli`
2. Start DB servers
3. Load MeTTa knowledge base
4. Start Attention Broker Service
5. Start Query Agent Service

First, you need to install the latest version of `das-cli` in your environment. Follow the instructions in the [toolbox repo](https://github.com/singnet/das-toolbox) to do that.

Let's start by setting up the environment, on your terminal run the following command:

<span style="color:red">*THE COMMANDS BELOW WILL CREATE FILES IN YOUR FILESYSTEM*</span>.
```
$ das-cli config set

# NOTE: Be sure to set Attention Broker's port to 37007
# For all other prompts, you can just hit ENTER

>>>
Enter Redis port [40020]: 
Is it a Redis cluster? [y/N]: 
Enter MongoDB port [40021]: 
Enter MongoDB username [admin]: 
Enter MongoDB password [admin]: 
Is it a MongoDB cluster? [y/N]: 
Enter Jupyter Notebook port [40019]: 
Enter the Attention Broker port [40001]: 37007 <--- HERE
Enter the Query Agent port [40002]: 
Enter the Link Creation Agent Server port [40003]: 
Enter the Link Creation Agent buffer file [/tmp/requests_buffer.bin]: 
Enter the Link Creation Agent request interval (in seconds) [1]: 
Enter the Link Creation Agent thread count [1]: 
Enter the Link Creation Agent default timeout (in seconds) [10]: 
Do you want to save links to a Metta file? [Y/n]: 
Do you want to save links to the database? [Y/n]: 
Enter the Inference Agent port [40004]: 
Enter the Evolution agent port [40005]: 
Configuration file saved -> /home/gontijo/.das
```

After that check the current configurations by running the following cell:

In [1]:
!das-cli config list

+------------------------------+--------------------+-------------------------------------------------------------------------+
| Service                      | Name               | Value                                                                   |
+------------------------------+--------------------+-------------------------------------------------------------------------+
|                              | schema_hash        | 7861d213288eec0b37e95fc770e5407258006ec617589a7a4d1bc8c32b457d0e        |
| services.redis               | port               | 40020                                                                   |
| services.redis               | container_name     | das-cli-redis-40020                                                     |
| services.redis               | cluster            | False                                                                   |
| services.redis               | nodes              | [{'context': 'default', 'ip': 'localhost', 'userna

Containers for the DBMS servers will be created listening on the given ports. Run the next cell to make sure any previously used containers are properly removed. If there are none, nothing will be done.

In [2]:
!das-cli db stop

Stopping Redis service...
The Redis service named das-cli-redis-40020 at localhost is already stopped.
Stopping MongoDB service...
The MongoDB service named das-cli-mongodb-40021 at localhost is already stopped.


Now we need to start the DBMS servers.

In [3]:
!das-cli db start

Starting Redis service...
Redis has started successfully on port 40020 at localhost, operating under the server user arturgontijo.
Starting MongoDB service...
MongoDB has started successfully on port 40021 at localhost, operating under the server user arturgontijo.


You can double check that the DB containers are in place listing the active docker containers.

In [4]:
!docker ps

CONTAINER ID   IMAGE                COMMAND                  CREATED          STATUS                    PORTS                                                                                              NAMES
330dd8cbdb39   mongo:6.0.13-jammy   "docker-entrypoint.s…"   46 seconds ago   Up 46 seconds (healthy)   0.0.0.0:40021->27017/tcp, [::]:40021->27017/tcp                                                    das-cli-mongodb-40021
f48d1391666d   redis:7.2.3-alpine   "docker-entrypoint.s…"   46 seconds ago   Up 46 seconds             0.0.0.0:40020->40020/tcp, [::]:40020->40020/tcp, 0.0.0.0:50020->50020/tcp, [::]:50020->50020/tcp   das-cli-redis-40020


You should see containers for Redis and MongoDB listening on the ports you defined in the config file.

Now we need to load a MeTTa file. You can use your own file here or run the next cell to download the same file we used in [this section](#remotedas).

In [5]:
!curl https://raw.githubusercontent.com/singnet/das-toolbox/refs/heads/master/das-cli/src/examples/data/animals.metta -o /tmp/animals.metta

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1529  100  1529    0     0  29030      0 --:--:-- --:--:-- --:--:-- 29403


You may want to change the path in the cell below to point to another file.

In [6]:
!das-cli metta load /tmp/animals.metta

das-cli-mongodb-40021 is running on port 40021
das-cli-redis-40020 is running on port 40020
Loading metta file /tmp/animals.metta...
Connecting to Redis at localhost:40020
Connecting to MongoDB at localhost:40021
Done.


You may call `das-cli metta load` multiple times loading different files. To clear the databases you can use `das-cli db restart`.

Start Attention Broker service:

In [7]:
!das-cli ab start

Starting Attention Broker service...
Attention Broker started on port 37007


Start Query Agent service:

In [8]:
!das-cli qa start

Starting Query Agent service...
Query Agent started on port 40002


In [9]:
import hyperon

metta = hyperon.MeTTa()
def run(program='!(+ 1 2)'):
    for result in metta.run(program):
        print(result)

In [10]:
run('!(import! &self das)')

[()]


In [11]:
run('!(bind! &das (new-das! (localhost:55000-55999) (localhost:40002)))')

[()]


From now on you can run queries on the `&das` space:

In [12]:
run('!(match &das (Similarity "human" $S) ($S))')

[("monkey"), ("ent"), ("chimp")]


In [13]:
run('!(match &das (and (Similarity "human" $S) (Inheritance $S "plant")) ($S))')

[("ent")]


Stop all services (removing their containers):

In [14]:
!das-cli qa stop
!das-cli ab stop
!das-cli db stop

Stopping Query Agent service...
Query Agent service stopped
Stopping Attention Broker service...
Attention Broker service stopped
Stopping Redis service...
The Redis service at localhost has been stopped by the server user arturgontijo
Stopping MongoDB service...
The MongoDB service at localhost has been stopped by the server user arturgontijo
